In [1]:
'''
Resnet for face recognition

MLAI Project

yidongh@andrew.cmu.edu

2019-11-28
'''
import os
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import time

import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.datasets
import torch.optim as optim

from torch.utils.data import DataLoader, Dataset

cuda = torch.cuda.is_available()
cuda

True

## ResNet

In [2]:
# Define the architecture
def conv3x3(in_planes, out_planes, stride=1, groups=1, dilation=1):
    return nn.Conv2d(in_planes, out_planes, kernel_size=3, stride=stride, padding=dilation,
                    groups=groups, bias = False, dilation=dilation)

def conv1x1(in_planes, out_planes, stride=1):
    return nn.Conv2d(in_planes, out_planes, kernel_size=1, stride=stride, bias=False)

class BasicBlock(nn.Module):
    expansion = 1
    __constants__ = ['downsample']
    
    def __init__(self, inplanes, planes, stride=1, downsample=None, groups=1,
                base_width=64, dilation=1, norm_layer = None):
        super(BasicBlock, self).__init__()
        if norm_layer is None:
            norm_layer = nn.BatchNorm2d
        if groups!=1 or base_width!=64:
            raise ValueError('BasicBlock only supports groups=1 and base_width=64')
        if dilation > 1:
            raise NotImplementedError("Dilation > 1 not supported in BasicBlock")
            
        # Both self.conv1 and self.downsample layers downsample the input when stride != 1
        self.conv1 = conv3x3(inplanes, planes, stride)
        self.bn1 = norm_layer(planes)
        self.relu = nn.ReLU(inplace=True) # Notice this operation. BatchNorm doesn't need its output to do back-prop. Bui conv2d needs.
        self.conv2 = conv3x3(planes,planes)
        self.bn2 = norm_layer(planes)
        self.downsample = downsample
        self.stride = stride
        
    def forward(self, x):
        identity = x
        
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        
        out = self.conv2(out)
        out = self.bn2(out)
        
        if self.downsample is not None:
            identity = self.downsample(x)
            
        out += identity
        out = self.relu(out)
        
        return out
    
class Bottleneck(nn.Module):
    expansion = 4
    __constants__ = ['downsample']
    
    def __init__(self, inplanes, planes, stride=1, downsample=None, groups=1,
                base_width=64, dilation=1, norm_layer=None):
        super(Bottleneck, self).__init__()
        if norm_layer is None:
            norm_layer = nn.BatchNorm2d
        width = int(planes * (base_width/64.)) * groups
        # Both self.conv2 and self.downsample layers downsample the input when stride != 1
        self.conv1 = conv1x1(inplanes, width)
        self.bn1 = norm_layer(width)
        self.conv2 = conv3x3(width, width, stride, groups, dilation)
        self.bn2 = norm_layer(width)
        self.conv3 = conv1x1(width,planes * self.expansion)
        self.bn3 = norm_layer(planes * self.expansion)
        self.relu = nn.ReLU(inplace=True)
        self.downsample = downsample
        self.stride = stride
        
    def forward(self, x):
        identity = x
        
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        
        out = self.conv2(out)
        out = self.bn2(out)
        out = self.relu(out)
        
        out = self.conv3(out)
        out = self.bn3(out)
        
        if self.downsample is not None:
            identity = self.downsample(x)
            
        out += identity
        out = self.relu(out)
        
        return out


In [3]:
class ResNet(nn.Module):
    def __init__(self, block, layers, num_classes = 2300, zero_init_residual = False,
                groups = 1, width_per_group = 64, replace_stride_with_dilation=None,
                norm_layer = None):
        super(ResNet, self).__init__()
        if norm_layer is None:
            norm_layer = nn.BatchNorm2d
        self._norm_layer = norm_layer
        
        self.inplanes = 64
        self.dilation = 1
        if replace_stride_with_dilation is None:
            replace_stride_with_dilation = [False, False, False]
        if len(replace_stride_with_dilation) != 3:
            raise ValueError("replace_stride_with_dilation should be none"
                            "or a 3-element tuple, got {}".format(replace_stride_with_dilation))
        self.groups = groups
        self.base_width = width_per_group
        
        # This could be modified. If stride = 2, then 32->16. Now I change it from 2 to 1.
        self.conv1 = nn.Conv2d(3, self.inplanes, kernel_size=1, stride=1, padding=0,
                              bias = False)
        self.bn1 = norm_layer(self.inplanes)
        self.relu = nn.ReLU(inplace = True)
        # Maxpool might be unnecessary
        #self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        
        self.layer1 = self._make_layer(block, 64, layers[0])
        self.layer2 = self._make_layer(block, 128, layers[1], stride=1, dilate=replace_stride_with_dilation[0])
        self.layer3 = self._make_layer(block, 256, layers[2], stride=2, dilate=replace_stride_with_dilation[1])
        self.layer4 = self._make_layer(block, 512, layers[3], stride=2, dilate=replace_stride_with_dilation[2])
        
        self.avgpool = nn.AdaptiveAvgPool2d((1,1))
        self.fc = nn.Linear(512 * block.expansion, num_classes)
        
        # Initialization
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
            elif isinstance(m, (nn.BatchNorm2d, nn.GroupNorm)):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)
                
        if zero_init_residual:
            for m in self.modules():
                if isinstance(m, Bottleneck):
                    nn.init.constant_(m.bn3.weight, 0)
                elif isinstance(m, BasicBlock):
                    nn.init.constant_(m.b2.weight, 0)
                    
    def _make_layer(self, block, planes, blocks, stride=1, dilate=False):
        norm_layer = self._norm_layer
        downsample = None
        previous_dilation = self.dilation
        if dilate:
            self.dilation *= stride
            stride = 1
        if stride!=1 or self.inplanes != planes*block.expansion:
            downsample = nn.Sequential(
            conv1x1(self.inplanes,planes*block.expansion,stride),
            norm_layer(planes*block.expansion))
            
        layers = []
        layers.append(block(self.inplanes, planes, stride, downsample, self.groups,
                            self.base_width, previous_dilation, norm_layer))
        
        self.inplanes = planes * block.expansion
        for _ in range(1, blocks):
            layers.append(block(self.inplanes, planes, groups=self.groups,
                                base_width=self.base_width, dilation=self.dilation,
                                norm_layer=norm_layer))
            
        return nn.Sequential(*layers)
    
    def _forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        # x = self.maxpool(x)
        #print('1',x.shape)
        
        x = self.layer1(x)
        #print('2',x.shape)
        x = self.layer2(x)
        #print('3',x.shape)
        x = self.layer3(x)
        #print('4',x.shape)
        x = self.layer4(x)
        #print('5',x.shape)
        
        x = self.avgpool(x)
        #print('6',x.shape)
        x = torch.flatten(x, 1)
        #x = self.fc(x)
        
        return x
        
    forward = _forward
    
def _resnet(arch, block, layers, **kwargs):
    model = ResNet(block, layers, **kwargs)
    return model

def resnet18(**kwargs):
    return _resnet('resnet18', BasicBlock, [2,2,2,2],**kwargs)

def resnet34(**kwargs):
    return _resnet('resnet34', BasicBlock, [3,4,6,3],**kwargs)

def resnet38(**kwargs):
    return _resnet('resnet38', Bottleneck, [3,3,3,3],**kwargs)

def resnet50(**kwargs):
    return _resnet('resnet50', Bottleneck, [3,4,6,3],**kwargs)

def resnet101(**kwargs):
    return _resnet('resnet101', Bottleneck, [3,4,23,3],**kwargs)

def resnet152(**kwargs):
    return _resnet('resnet152', Bottleneck, [3,8,36,3],**kwargs)

## Load Data

In [4]:
def parse_data(datadir):
    img_list = []
    ID_list = []
    uniqueID_list = []
    for root, directories, filenames in os.walk(datadir):
        b = -1
        for filename in filenames:
            if filename.endswith('.jpg'):
                if(filename[0]+filename[1]=='._'):
                    filename = filename[2:]
                filei = os.path.join(root, filename)
                img_list.append(filei)
                ID_list.append(root.split('/')[-1])
                a = root.split('/')[-1]
                if a!=b:
                    uniqueID_list.append(a)
                b=a
                
    class_n = len(uniqueID_list)
    target_dict = dict(zip(uniqueID_list, range(class_n)))
    reverse_dict = dict(zip(range(class_n),uniqueID_list))
    label_list = [target_dict[ID_key] for ID_key in ID_list]
    
    print('{}\t\t{}\n{}\t\t{}'.format('#Images', '#Labels', len(img_list), len(set(label_list))))
    return img_list, label_list, class_n, reverse_dict

## Load Network

In [6]:
network = resnet34()
network.load_state_dict(torch.load('Resnet34_iteration1_epoch2.pth'))
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
network.to(device)
network.eval()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): 

In [7]:
test_img_list  = []
for root, dictionaries, filenames in os.walk('test_classification/medium/'):
    for filename in filenames:
        img_path = os.path.join(root, filename)
        test_img_list.append(img_path)
        
test_img_list.sort()


In [8]:
print(len(test_img_list))

4600


In [9]:
class testDataset(Dataset):
    def __init__(self, file_list):
        self.file_list = file_list
    
    def __len__(self):
        return len(self.file_list)
    
    def __getitem__(self, index):
        img = Image.open(self.file_list[index])
        img = torchvision.transforms.ToTensor()(img)
        
        return img

In [10]:
testset = testDataset(test_img_list)
test_dataLoader = DataLoader(testset, batch_size=1, shuffle=False, num_workers=3)


In [11]:
print(len(testset))

4600


In [12]:
test_embedding_resnet = []
for batch_num, (feats) in enumerate(test_dataLoader):
    feats = feats.to(device)
    output = network(feats)
    embedding = np.copy(output.cpu().detach().numpy())
    test_embedding_resnet.append(embedding)
    
    del feats
    
    

In [13]:
test_embedding_save = np.zeros((1,512))
for i in range(len(test_embedding_resnet)):
    test_embedding_save = np.concatenate((test_embedding_save,test_embedding_resnet[i]),axis = 0)
    
test_embedding_save = test_embedding_save[1:,:]
print(test_embedding_save.shape)

(4600, 512)


In [14]:
print(len(test_embedding_resnet))

4600


In [15]:
np.save('resnet_test_embedding.npy',test_embedding_save)